<a href="https://colab.research.google.com/github/mcain368/Carifta-Analysis/blob/main/Cain_Michael_Hw_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

 # [9660] Homework # 4 - Random Forest
 Data file:
 * https://raw.githubusercontent.com/vjavaly/Baruch-CIS-STA-3920/main/data/temps_extended.csv

## Homework Submission Rules (for all homework assignments)
* Homework is due by 6:05 PM on the due date
  * No late submission will be accepted
* You must submit a cleanly executed notebook (*.ipynb)
  * Verify that you are submitting the correct homework file
* Homework file naming convention
  * LastName_FirstName_HwX.ipynb  [Replace X with the homework #]
    * 1 point deducted for submitting homework not complying with naming convention
* Before submission, execute "Kernel -> Restart Kernel and Run All Cells"
  * 1 point deducted for not submitting a cleanly executed notebook

## Homework #4 Requirements
* Part 1:
  * Load and examine data
  * Clean and prepare data for model training
  * Display RandomForestRegressor model default hyperparameters
  * Train RandomForestRegressor model with default hyperparameters
  * Print model accuracy
* Part 2: The objective is to improve performance above that of the default hyperparameters
  * If you used hyperparameter random_state in your initial model training, do NOT change this value during model retrainings
  * Do NOT re-split training and test sets during model retrainings
  * Prepare hyperparameter variables for random search
  * Setup and print random search hyperparameter variables grid
  * Setup and execute random search with k-fold cross validation using the hyperparameter variables grid
  * Print best hyperparameters combination
  * Print model accuracy from the best hyperparameters combination

In [ ]:
from datetime import datetime
print(f'Run time: {datetime.now().strftime("%D %T")}')

Run time: 04/18/24 06:23:27


### Import libraries

In [ ]:
import numpy as np
import pandas as pd
from pprint import pprint
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV

### Function to evaluate model accuracy
#### You will use this function twice in this notebook.
Please review the comments below carefully to find out when to invoke this function.

In [ ]:
def evaluate(model, model_string, test_features, test_labels):
    predictions = model.predict(test_features)
    errors = abs(predictions - test_labels)
    mape = 100 * np.mean(errors / test_labels)
    accuracy = 100 - mape
    print('Model Performance: {}'.format(model_string))
    print('Accuracy = {:0.2f}%.'.format(accuracy))

    return accuracy

## Part 1

### Load data

In [ ]:
df = pd.read_csv('https://raw.githubusercontent.com/vjavaly/Baruch-CIS-STA-3920/main/data/temps_extended.csv')

### Examine data

In [ ]:
df.shape

(2191, 12)

In [ ]:
df.head()

,year,month,day,weekday,ws_1,prcp_1,snwd_1,temp_2,temp_1,average,actual,friend
0,2011,1,1,Sat,4.92,0.00,0,36,37,45.6,40,40
1,2011,1,2,Sun,5.37,0.00,0,37,40,45.7,39,50
2,2011,1,3,Mon,6.26,0.00,0,40,39,45.8,42,42
3,2011,1,4,Tues,5.59,0.00,0,39,42,45.9,38,59
4,2011,1,5,Wed,3.80,0.03,0,42,38,46.0,45,39


### Clean up data

In [ ]:
# Drop unnecessary columns: year, month, day, weekday
df.drop(columns=['year','month','day','weekday'],inplace=True)

In [ ]:
# Display first few rows of updated dataframe
df.head()

,ws_1,prcp_1,snwd_1,temp_2,temp_1,average,actual,friend
0,4.92,0.00,0,36,37,45.6,40,40
1,5.37,0.00,0,37,40,45.7,39,50
2,6.26,0.00,0,40,39,45.8,42,42
3,5.59,0.00,0,39,42,45.9,38,59
4,3.80,0.03,0,42,38,46.0,45,39


### Separate independent variables and dependent variable
* Independent variables: all remaining variables except 'actual'
* Dependent variable: 'actual'

In [ ]:
X = df.drop('actual',axis=1)
y=df['actual']

### Split into training and test sets

In [ ]:
X_train,X_test,Y_train,Y_test= train_test_split(X,y,test_size=0.3,random_state=42)

### Instantiate the RandomForestRegressor model

In [ ]:
rf = RandomForestRegressor()

### Print RandomForestRegressor default hyperparameters

In [ ]:

print(rf.get_params())


{'bootstrap': True, 'ccp_alpha': 0.0, 'criterion': 'squared_error', 'max_depth': None, 'max_features': 1.0, 'max_leaf_nodes': None, 'max_samples': None, 'min_impurity_decrease': 0.0, 'min_samples_leaf': 1, 'min_samples_split': 2, 'min_weight_fraction_leaf': 0.0, 'n_estimators': 100, 'n_jobs': None, 'oob_score': False, 'random_state': None, 'verbose': 0, 'warm_start': False}


### Fit RandomForestRegressor model using the default hyperparameters

In [ ]:
rf.fit(X_train,Y_train)

RandomForestRegressor()

### Print accuracy for RandomForestRegressor model using the default hyperparameters
#### NOTE: Use "evaluate" function defined at top of this notebook.
For example, assuming the following variable values:
* model = rf
* model_string = 'using default hyperparameters'
* test_features = X_test
* test_labels = y_test

rfr_base_accuracy = evaluate(rf, 'With default hyperparameters', X_test, y_test)

In [ ]:

rfr_base_accuracy = evaluate(rf, 'using default hyperparameters', X_test, Y_test)


Model Performance: using default hyperparameters
Accuracy = 93.55%.


## Part 2

### NOTE: The objective of the hyperparameter search is to improve model performance above the default hyperparameters

### Prepare variables for hyperparameter search
* Using sklearn.ensemble.RandomForestRegressor documentation [https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestRegressor.html] choose at least 3 hyperparameters for random search
* For each hyperparameter selected, set up an array of values
  * For example: max_features = ['log2', 'sqrt']

In [ ]:

max_features = ['auto', 'sqrt', 'log2']
min_samples_split = [5, 10, 15]
max_depth = [5, 10, 15]
min_samples_leaf = [1, 2, 4]


### Create the hyperparameter grid for the random search
Use the variables prepared above

In [ ]:

param_grid = {
    'max_features': max_features,
    'min_samples_split': min_samples_split,
    'max_depth': max_depth,
    'min_samples_leaf': min_samples_leaf
}


### Print the hyperparameter grid for the random search

In [ ]:

print(param_grid)

{'max_features': ['auto', 'sqrt', 'log2'], 'min_samples_split': [5, 10, 15], 'max_depth': [5, 10, 15], 'min_samples_leaf': [1, 2, 4]}


### Set up random search with k-fold cross validation using the hyperparameter grid

In [ ]:
# Random search of hyperparameters using 5 fold cross validation.

rf_random = RandomizedSearchCV(estimator=rf, param_distributions=param_grid, n_iter=100, cv=5, verbose=1, random_state=42, n_jobs=-1)

### Fit the random search model
Be patient, this might take a minute or longer

In [ ]:
rf_random.fit(X_train, Y_train)

Fitting 5 folds for each of 81 candidates, totalling 405 fits


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_search.py:305: UserWarning: The total space of parameters 81 is smaller than n_iter=100. Running 81 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_forest.py:413: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0` or remove this parameter as it is also the default value for RandomForestRegressors and ExtraTreesRegressors.
  warn(


RandomizedSearchCV(cv=5, estimator=RandomForestRegressor(), n_iter=100,
                   n_jobs=-1,
                   param_distributions={'max_depth': [5, 10, 15],
                                        'max_features': ['auto', 'sqrt',
                                                         'log2'],
                                        'min_samples_leaf': [1, 2, 4],
                                        'min_samples_split': [5, 10, 15]},
                   random_state=42, verbose=1)

### Print the best hyperparameters found by the random search

In [ ]:
print(rf_random.best_params_)

{'min_samples_split': 10, 'min_samples_leaf': 2, 'max_features': 'auto', 'max_depth': 5}


### Print best random search model accuracy
#### NOTE: Use "evaluate" function defined at top of this notebook.

In [ ]:
rfr_base_accuracy = evaluate(rf_random, 'using default hyperparameters', X_test, Y_test)

Model Performance: using default hyperparameters
Accuracy = 93.80%.
